Данный ноутбук содержит код для создания категориальных и lag-фичей, подготовки данных к последующему этапу ML и собственно эксперименты ML c моделью линейной регрессии и моделью Catboost на основе timeseries полученного для станции STA-DK0034A Дании.

<a id='part_0'></a>

## Содержание
### [1.Подготовка данных к ML](#part_1)
### [2.Эксперименты с моделями линейной регрессии](#part_2)
### [3.Эксперименты с Catboost](#part_3)
### [4.Обсуждение и выводы](#part_4)

<a id='part_1'></a>
## 1.Подготовка данных к ML

[ВЕРНУТЬСЯ В НАЧАЛО](#part_0)

In [1]:
import datetime
import pytz
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import pandas as pd
pd.set_option('display.max_columns', None)
np.random.seed(25)

In [2]:
### Load previously created final AQI timeseries for 'STA-DK0034A' station in Denmark

ts_final = pd.read_csv('ts_final_dk.csv', low_memory=False)
ts_final['DatetimeEnd'] = pd.to_datetime(ts_final['DatetimeEnd'], format="%Y-%m-%d %H:%M:%S")
ts_final = ts_final.set_index('DatetimeEnd')

In [3]:
### Check that ts_final is in a proper timeseries format

ts_final.head()

,aqi
DatetimeEnd,
2013-01-01 00:00:00+01:00,21.0
2013-01-02 00:00:00+01:00,41.0
2013-01-03 00:00:00+01:00,39.0
2013-01-04 00:00:00+01:00,48.0
2013-01-05 00:00:00+01:00,33.0


In [4]:
### Functions to generate categorical and lag-features

def feature_creator_v1(df_temp,number_of_lag_features=10): # Generate only numeric features
    df = df_temp.copy()
    df.columns = ['aqi_d0']
    n_features = int(number_of_lag_features)
    current_column_name = 'aqi_d0'
    prev_column_name = 'aqi_d0'
    iternum = 1
    for i in range(n_features):
        current_column_name = current_column_name[:5] + str(iternum)
        df[current_column_name] = df[prev_column_name].shift(1)
        iternum +=1
        prev_column_name = current_column_name
    return df

def feature_creator_v2(df_temp,number_of_lag_features=10): # Generate both categorical and numeric features
    df = df_temp.copy()
    df.columns = ['aqi_d0']
    df['month'] = df.index.month
    df['monthday'] = df.index.strftime("%d")
    df['weekday'] = df.index.weekday
    weekdays_list = list(df["weekday"])
    weekend_list = []
    for i in range(len(weekdays_list)):
        weekend_flag = False
        if (weekdays_list[i] == 6) | (weekdays_list[i] == 0):
            weekend_flag = True
        weekend_list.append(weekend_flag)
    df['weekend'] = weekend_list
    n_features = int(number_of_lag_features)
    current_column_name = 'aqi_d0'
    prev_column_name = 'aqi_d0'
    iternum = 1
    for i in range(n_features):
        current_column_name = current_column_name[:5] + str(iternum)
        df[current_column_name] = df[prev_column_name].shift(1)
        iternum +=1
        prev_column_name = current_column_name
    return df

In [5]:
### Check feature creators to make sure they work as expected

tsf1 = feature_creator_v1(ts_final, 4)
tsf2 = feature_creator_v2(ts_final, 4)
display(tsf1.tail())
display(tsf2.tail())

,aqi_d0,aqi_d1,aqi_d2,aqi_d3,aqi_d4
DatetimeEnd,,,,,
2022-10-26 00:00:00+01:00,33.0,32.0,22.0,18.0,24.0
2022-10-27 00:00:00+01:00,37.0,33.0,32.0,22.0,18.0
2022-10-28 00:00:00+01:00,43.0,37.0,33.0,32.0,22.0
2022-10-29 00:00:00+01:00,35.0,43.0,37.0,33.0,32.0
2022-10-30 00:00:00+01:00,34.0,35.0,43.0,37.0,33.0


,aqi_d0,month,monthday,weekday,weekend,aqi_d1,aqi_d2,aqi_d3,aqi_d4
DatetimeEnd,,,,,,,,,
2022-10-26 00:00:00+01:00,33.0,10,26,2,False,32.0,22.0,18.0,24.0
2022-10-27 00:00:00+01:00,37.0,10,27,3,False,33.0,32.0,22.0,18.0
2022-10-28 00:00:00+01:00,43.0,10,28,4,False,37.0,33.0,32.0,22.0
2022-10-29 00:00:00+01:00,35.0,10,29,5,False,43.0,37.0,33.0,32.0
2022-10-30 00:00:00+01:00,34.0,10,30,6,True,35.0,43.0,37.0,33.0


In [6]:
### Create 2 datasets; one with 10 lag-features, another with 30 lag-features

tsf10 = feature_creator_v1(ts_final, 10)
tsf30 = feature_creator_v1(ts_final, 30)
display(len(tsf10))
display(len(tsf30))

3590

3590

In [7]:
### Truncate first 30 rows of created datasets to remove rows containing NaN values

tsf10 = tsf10.iloc[30:,:]
tsf30 = tsf30.iloc[30:,:]

In [8]:
### Verify that we truncated exactly 30 rows

display(len(tsf10))
display(len(tsf30))

3560

3560

In [9]:
### Function to split dataset into train and test parts

def data_splitter(df_temp):
    df = df_temp
    df_train = df[df.index < "2022-01-01 00:00:00+01:00"].copy()
    df_test = df[df.index >= "2022-01-01 00:00:00+01:00"].copy()
    return df_train, df_test


### Function to split dataset into train and test parts v2

def data_splitter_v2(df_temp):
    df = df_temp
    df_train = df[df.index < "2021-10-01 00:00:00+01:00"].copy()
    df_test = df[df.index >= "2021-10-01 00:00:00+01:00"].copy()
    return df_train, df_test

<a id='part_2'></a>
## 2.Эксперименты с моделями линейной регрессии

[ВЕРНУТЬСЯ В НАЧАЛО](#part_0)

In [10]:
### Apply previously created function to our datasets

df_train10, df_test10 = data_splitter(tsf10)
df_train30, df_test30 = data_splitter(tsf30)

In [11]:
### Split datasets further into X(features) and y(target) components

columns_to_drop = ['aqi_d0']

y_train10 = df_train10['aqi_d0']
X_train10 = df_train10.drop(columns_to_drop, axis=1)
y_test10 = df_test10['aqi_d0']
X_test10 = df_test10.drop(columns_to_drop, axis=1)

y_train30 = df_train30['aqi_d0']
X_train30 = df_train30.drop(columns_to_drop, axis=1)
y_test30 = df_test30['aqi_d0']
X_test30 = df_test30.drop(columns_to_drop, axis=1)

In [12]:
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error as MSE
from sklearn.metrics import mean_absolute_error as MAE

In [13]:
### Standard linear regression with 10 features

scaler = StandardScaler()
scaler.fit(X_train10)

X_train_scaled = scaler.transform(X_train10)
X_test_scaled = scaler.transform(X_test10)

lr_scaled = LinearRegression()
lr_scaled.fit(X_train_scaled, y_train10)
pred_train10 = lr_scaled.predict(X_train_scaled)
pred_test10 = lr_scaled.predict(X_test_scaled)

print('for TRAIN:')
print('MSE:', MSE(y_train10, pred_train10))
print('R2:', r2_score(y_train10, pred_train10))
print('for TEST:')
print('MSE:', MSE(y_test10, pred_test10))
print('R2:', r2_score(y_test10, pred_test10))

for TRAIN:
MSE: 94.32011222958425
R2: 0.40597959933723293
for TEST:
MSE: 59.3182458129779
R2: 0.28025522634722955


In [14]:
### Cross-validation for linear regression with 10 numeric features

from sklearn.model_selection import TimeSeriesSplit

tscv = TimeSeriesSplit(n_splits = 10)
rmse = []
mae = []
r2 = []
rmse_dfm = []
mae_dfm = []
r2_dfm = []
all_predictions = None
aqi_mean = tsf10['aqi_d0'].mean()
for train_index, test_index in tscv.split(tsf10):
    cv_train, cv_test = tsf10.iloc[train_index], tsf10.iloc[test_index]
    
    y_cv_train = cv_train['aqi_d0']
    X_cv_train = cv_train.drop('aqi_d0', axis=1)
    y_cv_test = cv_test['aqi_d0']
    X_cv_test = cv_test.drop('aqi_d0', axis=1)
    
    scaler = StandardScaler()
    scaler.fit(X_cv_train)
    
    X_cv_train_scaled = scaler.transform(X_cv_train)
    X_cv_test_scaled = scaler.transform(X_cv_test)
    
    lr_scaled = LinearRegression()
    lr_scaled.fit(X_cv_train_scaled, y_cv_train )
    predictions = lr_scaled.predict(X_cv_test_scaled)
 
    rmse.append(np.sqrt(MSE(y_cv_test, predictions)))
    mae.append(MAE(y_cv_test, predictions))
    r2.append(r2_score(y_cv_test, predictions))

    dfmean = pd.DataFrame(index=cv_test.index)
    dfmean['aqi_d0'] = aqi_mean
    
    rmse_dfm.append(np.sqrt(MSE(y_cv_test, dfmean.values)))
    mae_dfm.append(MAE(y_cv_test, dfmean.values))
    r2_dfm.append(r2_score(y_cv_test, dfmean.values))


rmse_df_mean = np.mean(rmse_dfm)
rmse_mean = np.mean(rmse)

mae_df_mean = np.mean(mae_dfm)
mae_mean = np.mean(mae)

r2_df_mean = np.mean(r2_dfm)
r2_mean = np.mean(r2)

In [15]:
print('Mean baseline:')
print('RMSE:', rmse_df_mean)
print('MAE:', mae_df_mean)
print('R2', r2_df_mean)
print('')
print('Linear regression with 10 lag-features:')
print('RMSE:', rmse_mean)
print('MAE:', mae_mean)
print('R2', r2_mean)

Mean baseline:
RMSE: 12.259450105194293
MAE: 8.964555605802344
R2 -0.06014788401808879

Linear regression with 10 lag-features:
RMSE: 9.604746175563356
MAE: 6.996398468789235
R2 0.3482510123808328


In [16]:
### Standard linear regression with 30 features

scaler = StandardScaler()
scaler.fit(X_train30)

X_train_scaled = scaler.transform(X_train30)
X_test_scaled = scaler.transform(X_test30)

lr_scaled = LinearRegression()
lr_scaled.fit(X_train_scaled, y_train30)
pred_train30 = lr_scaled.predict(X_train_scaled)
pred_test30 = lr_scaled.predict(X_test_scaled)

print('for TRAIN:')
print('MSE:', MSE(y_train30, pred_train30))
print('R2:', r2_score(y_train30, pred_train30))
print('for TEST:')
print('MSE:', MSE(y_test30, pred_test30))
print('R2:', r2_score(y_test30, pred_test30))

for TRAIN:
MSE: 93.31503263325668
R2: 0.4123095089439479
for TEST:
MSE: 59.16578889665561
R2: 0.2821050799163345


In [17]:
### Cross-validation for linear regression with 30 numeric features

from sklearn.model_selection import TimeSeriesSplit

tscv = TimeSeriesSplit(n_splits = 10)
rmse = []
mae = []
r2 = []
rmse_dfm = []
mae_dfm = []
r2_dfm = []
all_predictions = None
aqi_mean = tsf30['aqi_d0'].mean()
for train_index, test_index in tscv.split(tsf30):
    cv_train, cv_test = tsf30.iloc[train_index], tsf30.iloc[test_index]
    
    y_cv_train = cv_train['aqi_d0']
    X_cv_train = cv_train.drop('aqi_d0', axis=1)
    y_cv_test = cv_test['aqi_d0']
    X_cv_test = cv_test.drop('aqi_d0', axis=1)
    
    scaler = StandardScaler()
    scaler.fit(X_cv_train)
    
    X_cv_train_scaled = scaler.transform(X_cv_train)
    X_cv_test_scaled = scaler.transform(X_cv_test)
    
    lr_scaled = LinearRegression()
    lr_scaled.fit(X_cv_train_scaled, y_cv_train )
    predictions = lr_scaled.predict(X_cv_test_scaled)
 
    rmse.append(np.sqrt(MSE(y_cv_test, predictions)))
    mae.append(MAE(y_cv_test, predictions))
    r2.append(r2_score(y_cv_test, predictions))

    dfmean = pd.DataFrame(index=cv_test.index)
    dfmean['aqi_d0'] = aqi_mean
    
    rmse_dfm.append(np.sqrt(MSE(y_cv_test, dfmean.values)))
    mae_dfm.append(MAE(y_cv_test, dfmean.values))
    r2_dfm.append(r2_score(y_cv_test, dfmean.values))


rmse_df_mean = np.mean(rmse_dfm)
rmse_mean = np.mean(rmse)

mae_df_mean = np.mean(mae_dfm)
mae_mean = np.mean(mae)

r2_df_mean = np.mean(r2_dfm)
r2_mean = np.mean(r2)

In [18]:
print('Mean baseline:')
print('RMSE:', rmse_df_mean)
print('MAE:', mae_df_mean)
print('R2', r2_df_mean)
print('')
print('Linear regression with 30 lag-features:')
print('RMSE:', rmse_mean)
print('MAE:', mae_mean)
print('R2', r2_mean)

Mean baseline:
RMSE: 12.259450105194293
MAE: 8.964555605802344
R2 -0.06014788401808879

Linear regression with 30 lag-features:
RMSE: 9.669484518164268
MAE: 7.033486317492153
R2 0.33962362863053464


Как при использовании 10, так и при использовании 30 предыдущих индексов AQI, получаются одинаково плохие результаты. Посмотрим что будет при использовании 100 предыдущих значений.

In [19]:
### Standard linear regression with 100 numeric features

tsf100 = feature_creator_v1(ts_final, 100)
tsf100 = tsf100.iloc[100:,:]
df_train100, df_test100 = data_splitter(tsf100)
y_train100 = df_train100['aqi_d0']
X_train100 = df_train100.drop(columns_to_drop, axis=1)
y_test100 = df_test100['aqi_d0']
X_test100 = df_test100.drop(columns_to_drop, axis=1)

scaler = StandardScaler()
scaler.fit(X_train100)

X_train_scaled = scaler.transform(X_train100)
X_test_scaled = scaler.transform(X_test100)

lr_scaled = LinearRegression()
lr_scaled.fit(X_train_scaled, y_train100)
pred_train100 = lr_scaled.predict(X_train_scaled)
pred_test100 = lr_scaled.predict(X_test_scaled)

print('for TRAIN:')
print('MSE:', MSE(y_train100, pred_train100))
print('R2:', r2_score(y_train100, pred_train100))
print('for TEST:')
print('MSE:', MSE(y_test100, pred_test100))
print('R2:', r2_score(y_test100, pred_test100))

for TRAIN:
MSE: 89.96312397206802
R2: 0.4285474809910045
for TEST:
MSE: 61.105824230204
R2: 0.25856543755358496


C:\Users\Mint\AppData\Local\Temp\ipykernel_16572\257147072.py:12: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[current_column_name] = df[prev_column_name].shift(1)


In [20]:
### Cross-validation for linear regression with 100 numeric features

from sklearn.model_selection import TimeSeriesSplit

tscv = TimeSeriesSplit(n_splits = 10)
rmse = []
mae = []
r2 = []
rmse_dfm = []
mae_dfm = []
r2_dfm = []
all_predictions = None
aqi_mean = tsf100['aqi_d0'].mean()
for train_index, test_index in tscv.split(tsf100):
    cv_train, cv_test = tsf100.iloc[train_index], tsf100.iloc[test_index]
    
    y_cv_train = cv_train['aqi_d0']
    X_cv_train = cv_train.drop('aqi_d0', axis=1)
    y_cv_test = cv_test['aqi_d0']
    X_cv_test = cv_test.drop('aqi_d0', axis=1)
    
    scaler = StandardScaler()
    scaler.fit(X_cv_train)
    
    X_cv_train_scaled = scaler.transform(X_cv_train)
    X_cv_test_scaled = scaler.transform(X_cv_test)
    
    lr_scaled = LinearRegression()
    lr_scaled.fit(X_cv_train_scaled, y_cv_train )
    predictions = lr_scaled.predict(X_cv_test_scaled)
 
    rmse.append(np.sqrt(MSE(y_cv_test, predictions)))
    mae.append(MAE(y_cv_test, predictions))
    r2.append(r2_score(y_cv_test, predictions))

    dfmean = pd.DataFrame(index=cv_test.index)
    dfmean['aqi_d0'] = aqi_mean
    
    rmse_dfm.append(np.sqrt(MSE(y_cv_test, dfmean.values)))
    mae_dfm.append(MAE(y_cv_test, dfmean.values))
    r2_dfm.append(r2_score(y_cv_test, dfmean.values))


rmse_df_mean = np.mean(rmse_dfm)
rmse_mean = np.mean(rmse)

mae_df_mean = np.mean(mae_dfm)
mae_mean = np.mean(mae)

r2_df_mean = np.mean(r2_dfm)
r2_mean = np.mean(r2)

In [21]:
print('Mean baseline:')
print('RMSE:', rmse_df_mean)
print('MAE:', mae_df_mean)
print('R2', r2_df_mean)
print('')
print('Linear regression with 100 lag-features:')
print('RMSE:', rmse_mean)
print('MAE:', mae_mean)
print('R2', r2_mean)

Mean baseline:
RMSE: 12.134831448276817
MAE: 8.860987770375926
R2 -0.06503923953351068

Linear regression with 100 lag-features:
RMSE: 10.027423257130128
MAE: 7.40013383225579
R2 0.27212223754528236


Чем больше lag-фичей мы используем, тем хуже становится R2 score, и тем выше становятся RMSE и MAE. Одна из возможных причин таких результатов, это то что мы используем значения индексов усредненные по дням, а не по часам. Возможно, в случае часовых AQI результаты были бы немного лучше. Но на данном этапе мы пока не будем создавать новый датасет с часовыми индексами, а продолжим работать на текущем датасете. Попробуем использовать полиномиальные фичи и Lasso-регрессию для датасета с 30 lag-фичами, возможно это позволит улучшить результаты предсказания.

In [22]:
from sklearn.linear_model import Lasso
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import PolynomialFeatures

In [23]:
### Check prediction quality of basic Lasso-regression

scaler = StandardScaler()
scaler.fit(X_train30)

X_train_scaled = scaler.transform(X_train30)
X_test_scaled = scaler.transform(X_test30)
lasso = Lasso()
lasso.fit(X_train_scaled, y_train30)
pred_lasso_train30 = lasso.predict(X_train_scaled)
pred_lasso_test30 = lasso.predict(X_test_scaled)

print('for TRAIN:')
print('MSE:', MSE(y_train30, pred_lasso_train30))
print('R2:', r2_score(y_train30, pred_lasso_train30))
print('for TEST:')
print('MSE:', MSE(y_test30, pred_lasso_test30))
print('R2:', r2_score(y_test30, pred_lasso_test30))

for TRAIN:
MSE: 96.46502926519346
R2: 0.3924710861817393
for TEST:
MSE: 61.801754987897304
R2: 0.25012128147969825


Тут практически то же самое что и для обычной регрессии. Добавим полиномиальные фичи и посмотрим улучшит ли это как-нибудь наши результаты.

In [24]:
### Pipeline for Lasso-regression with polynomial features

pipeline_Lasso = Pipeline([('scaler', StandardScaler()),
                           ('polyfeatures', PolynomialFeatures(degree = 2)),
                           ('lasso', Lasso())])

In [25]:
### Check Lasso regression with polynomial features

pipeline_Lasso.fit(X_train30, y_train30)
pred_lasso_poly_train30 = pipeline_Lasso.predict(X_train30)
pred_lasso_poly_test30 = pipeline_Lasso.predict(X_test30)

print('for TRAIN:')
print('MSE:', MSE(y_train30, pred_lasso_poly_train30))
print('R2:', r2_score(y_train30, pred_lasso_poly_train30))
print('for TEST:')
print('MSE:', MSE(y_test30, pred_lasso_poly_test30))
print('R2:', r2_score(y_test30, pred_lasso_poly_test30))

for TRAIN:
MSE: 96.29653576592746
R2: 0.3935322445452857
for TEST:
MSE: 61.99834915200721
R2: 0.2477358835265142


Полиномиальные признаки никак не улучшили результаты предсказания. Напоследок проверим улучшится ли качество предсказания при использовании категориальных фичей для датасета с 30-ю lag-фичами.

In [26]:
tsf30 = feature_creator_v2(ts_final, 30)
tsf30 = tsf30.iloc[30:,:]

In [27]:
### Check categorical features

categorical_features = tsf30.select_dtypes(exclude='number').columns.tolist()
print(categorical_features)

['monthday', 'weekend']


In [28]:
### Check numeric features

numerical_features = tsf30.select_dtypes(include='number').columns.tolist()
print(numerical_features)

['aqi_d0', 'month', 'weekday', 'aqi_d1', 'aqi_d2', 'aqi_d3', 'aqi_d4', 'aqi_d5', 'aqi_d6', 'aqi_d7', 'aqi_d8', 'aqi_d9', 'aqi_d10', 'aqi_d11', 'aqi_d12', 'aqi_d13', 'aqi_d14', 'aqi_d15', 'aqi_d16', 'aqi_d17', 'aqi_d18', 'aqi_d19', 'aqi_d20', 'aqi_d21', 'aqi_d22', 'aqi_d23', 'aqi_d24', 'aqi_d25', 'aqi_d26', 'aqi_d27', 'aqi_d28', 'aqi_d29', 'aqi_d30']


Две категориальные фичи попали в числовые. Сконвертируем их в другой формат, а затем разделим выборку на train и test. А еще немного увеличим тестовую выборку, чтобы она включала все возможные категории.

In [29]:
### Prepare data for subsequent preproccessing

tsf30['month'] = tsf30['month'].astype(str)
tsf30['weekday'] = tsf30['weekday'].astype(str)

df_train30, df_test30 = data_splitter_v2(tsf30)
y_train30 = df_train30['aqi_d0']
X_train30 = df_train30.drop(columns_to_drop, axis=1)
y_test30 = df_test30['aqi_d0']
X_test30 = df_test30.drop(columns_to_drop, axis=1)

categorical_features = X_train30.select_dtypes(exclude='number').columns.tolist()
numerical_features = X_train30.select_dtypes(include='number').columns.tolist()

In [30]:
### Create numeric and categorical pipelines

from sklearn.preprocessing import OneHotEncoder

numeric_pipeline = Pipeline(steps=[('scale', StandardScaler())])

categorical_pipeline = Pipeline(steps=[('one-hot', OneHotEncoder(handle_unknown='ignore', sparse=False))])

In [31]:
### Combine both previously created pipelines into one

from sklearn.compose import ColumnTransformer

full_processor = ColumnTransformer(transformers=[('numeric', numeric_pipeline, numerical_features),
                                                 ('categorical', categorical_pipeline, categorical_features)])

In [32]:
### Check prediction quality of Lasso with categorical features

lasso = Lasso()
lasso.fit(full_processor.fit_transform(X_train30), y_train30)
pred_lasso_train30 = lasso.predict(full_processor.fit_transform(X_train30))
pred_lasso_test30 = lasso.predict(full_processor.fit_transform(X_test30))

print('for TRAIN:')
print('MSE:', MSE(y_train30, pred_lasso_train30))
print('R2:', r2_score(y_train30, pred_lasso_train30))
print('for TEST:')
print('MSE:', MSE(y_test30, pred_lasso_test30))
print('R2:', r2_score(y_test30, pred_lasso_test30))

for TRAIN:
MSE: 96.04226664132453
R2: 0.39263507881091897
for TEST:
MSE: 97.30865324545069
R2: -0.01455529759846863


Стало только хуже. Наверное на этом можно закончить эксперименты с линейными моделями и перейти к более сложным моделям, так как маловероятно что здесь можно как-то существенным образом улучшить результаты. В качестве следующей модели возьмем Catboost. 

Наилучшие результаты были получены для модели простой линейной регрессии с 10 lag-фичами без категориальных признаков:<br>
RMSE: 9.604746175563356<br>
MAE: 6.996398468789235<br>
R2 0.3482510123808328<br>

<a id='part_3'></a>
## 3.Эксперименты с Catboost

[ВЕРНУТЬСЯ В НАЧАЛО](#part_0)

In [33]:
from catboost import CatBoostClassifier, Pool, metrics, cv
from catboost import CatBoostRegressor
from sklearn.metrics import accuracy_score

In [34]:
### Create timeseries with 30 lag-features and categorical features

tsf30 = feature_creator_v2(ts_final, 30)
tsf30['month'] = tsf30['month'].astype(str)
tsf30['weekday'] = tsf30['weekday'].astype(str)

In [35]:
### Split data into train and test; also specify categorical columns

df_train30, df_test30 = data_splitter_v2(tsf30)
y_train30 = df_train30['aqi_d0']
X_train30 = df_train30.drop(columns_to_drop, axis=1)
y_test30 = df_test30['aqi_d0']
X_test30 = df_test30.drop(columns_to_drop, axis=1)
categorical_features_indices = np.where(X_train30.dtypes != float)[0]

In [36]:
### Initialize CatboostRegressor

model = CatBoostRegressor(loss_function='RMSE',
                           random_seed=25,
                           logging_level='Silent')

In [37]:
### Train CatboostRegressor

model.fit(X_train30, y_train30,
          cat_features=categorical_features_indices,
          eval_set=(X_test30, y_test30),
          #     logging_level='Verbose',  # you can uncomment this for text output
          plot=True
         )

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

In [38]:
### Check results of Catboost experiment

preds = model.predict(X_test30)

print('RMSE:', np.sqrt(MSE(y_test30, preds)))
print('MAE:', MAE(y_test30, preds))
print('R2:', r2_score(y_test30, preds))

RMSE: 8.285007842746582
MAE: 5.950441980702406
R2: 0.28433445556342474


К сожалению, чуда не случилось и catboost показывает примерно такие же результаты как и простая линейная регрессия.

In [39]:
### Cross-validation for Catboost with 30 numeric features and 4 categorical features

from sklearn.model_selection import TimeSeriesSplit

tscv = TimeSeriesSplit(n_splits = 10)
rmse = []
mae = []
r2 = []
rmse_dfm = []
mae_dfm = []
r2_dfm = []
all_predictions = None
aqi_mean = tsf30['aqi_d0'].mean()
for train_index, test_index in tscv.split(tsf30):
    cv_train, cv_test = tsf30.iloc[train_index], tsf30.iloc[test_index]
    
    y_cv_train = cv_train['aqi_d0']
    X_cv_train = cv_train.drop('aqi_d0', axis=1)
    y_cv_test = cv_test['aqi_d0']
    X_cv_test = cv_test.drop('aqi_d0', axis=1)
    
    model = CatBoostRegressor(loss_function='RMSE',
                              random_seed=25,
                              logging_level='Silent')

    model.fit(X_cv_train, y_cv_train, cat_features=categorical_features_indices, eval_set=(X_cv_test, y_cv_test))
    predictions = model.predict(X_cv_test)
 
    rmse.append(np.sqrt(MSE(y_cv_test, predictions)))
    mae.append(MAE(y_cv_test, predictions))
    r2.append(r2_score(y_cv_test, predictions))

    dfmean = pd.DataFrame(index=cv_test.index)
    dfmean['aqi_d0'] = aqi_mean
    
    rmse_dfm.append(np.sqrt(MSE(y_cv_test, dfmean.values)))
    mae_dfm.append(MAE(y_cv_test, dfmean.values))
    r2_dfm.append(r2_score(y_cv_test, dfmean.values))


rmse_df_mean = np.mean(rmse_dfm)
rmse_mean = np.mean(rmse)

mae_df_mean = np.mean(mae_dfm)
mae_mean = np.mean(mae)

r2_df_mean = np.mean(r2_dfm)
r2_mean = np.mean(r2)

In [40]:
print('Mean baseline:')
print('RMSE:', rmse_df_mean)
print('MAE:', mae_df_mean)
print('R2', r2_df_mean)
print('')
print('Catboost with 30 numeric and 4 categorical features:')
print('RMSE:', rmse_mean)
print('MAE:', mae_mean)
print('R2', r2_mean)

Mean baseline:
RMSE: 12.27240446713256
MAE: 8.978707042397936
R2 -0.05862792066997431

Catboost with 30 numeric and 4 categorical features:
RMSE: 9.66646053216206
MAE: 7.0764718227070365
R2 0.34738229879250104


In [41]:
### Cross-validation for Catboost with 30 numeric features

tsf30 = feature_creator_v1(ts_final, 30)
df_train30, df_test30 = data_splitter_v2(tsf30)
y_train30 = df_train30['aqi_d0']
X_train30 = df_train30.drop(columns_to_drop, axis=1)
y_test30 = df_test30['aqi_d0']
X_test30 = df_test30.drop(columns_to_drop, axis=1)
categorical_features_indices = np.where(X_train30.dtypes != float)[0]

tscv = TimeSeriesSplit(n_splits = 10)
rmse = []
mae = []
r2 = []
rmse_dfm = []
mae_dfm = []
r2_dfm = []
all_predictions = None
aqi_mean = tsf30['aqi_d0'].mean()
for train_index, test_index in tscv.split(tsf30):
    cv_train, cv_test = tsf30.iloc[train_index], tsf30.iloc[test_index]
    
    y_cv_train = cv_train['aqi_d0']
    X_cv_train = cv_train.drop('aqi_d0', axis=1)
    y_cv_test = cv_test['aqi_d0']
    X_cv_test = cv_test.drop('aqi_d0', axis=1)
    
    model = CatBoostRegressor(loss_function='RMSE',
                              random_seed=25,
                              logging_level='Silent')

    model.fit(X_cv_train, y_cv_train, cat_features=categorical_features_indices, eval_set=(X_cv_test, y_cv_test))
    predictions = model.predict(X_cv_test)
 
    rmse.append(np.sqrt(MSE(y_cv_test, predictions)))
    mae.append(MAE(y_cv_test, predictions))
    r2.append(r2_score(y_cv_test, predictions))

    dfmean = pd.DataFrame(index=cv_test.index)
    dfmean['aqi_d0'] = aqi_mean
    
    rmse_dfm.append(np.sqrt(MSE(y_cv_test, dfmean.values)))
    mae_dfm.append(MAE(y_cv_test, dfmean.values))
    r2_dfm.append(r2_score(y_cv_test, dfmean.values))


rmse_df_mean = np.mean(rmse_dfm)
rmse_mean = np.mean(rmse)

mae_df_mean = np.mean(mae_dfm)
mae_mean = np.mean(mae)

r2_df_mean = np.mean(r2_dfm)
r2_mean = np.mean(r2)

In [42]:
print('Mean baseline:')
print('RMSE:', rmse_df_mean)
print('MAE:', mae_df_mean)
print('R2', r2_df_mean)
print('')
print('Catboost with 30 numeric features:')
print('RMSE:', rmse_mean)
print('MAE:', mae_mean)
print('R2', r2_mean)

Mean baseline:
RMSE: 12.27240446713256
MAE: 8.978707042397936
R2 -0.05862792066997431

Catboost with 30 numeric features:
RMSE: 9.797108923677888
MAE: 7.183606290906108
R2 0.330641396660838


<a id='part_4'></a>
## 4.Обсуждение и выводы

[ВЕРНУТЬСЯ В НАЧАЛО](#part_0)

Ниже приведено сравнение результатов кросс-валидации для простой линейной регрессии и Catboost.

In [43]:
from IPython.core.display import HTML
table_css = 'table {align:left;display:block} '
HTML('<style>{}</style>'.format(table_css))

Metrics |LinearRegression |Catboost with cat.features|Catboost numeric only|Mean Baseline|
-----------|-----------|-----------|---------| ---------| 
RMSE|9.604746175563356|9.66646053216206|9.797108923677888| 12.27240446713256|
MAE|6.996398468789235|7.0764718227070365|7.183606290906108| 8.978707042397936|
R2 score|0.3482510123808328|0.34738229879250104|0.330641396660838| -0.05862792066997431|

Результаты, полученные в ходе данных экспериментов сложно назвать удовлетворительными. В целом, как линейная регрессия, так и Catboost показали примерно одинаковые результаты. Одинаково плохие. К сожалению ни один из способов использованных в данной работе не помог существенным образом улучшить качество предсказания. И это можно объяснить несколькими причинами: <br>
1) Мы использовали модели неспецифичные к временным рядам. В целом даже если смотреть публикации по Machine Learning за последние несколько лет, то в большинстве случаев для временных рядов используют либо специфичные к временным рядам модели, такие как ARIMA, либо Deep-Learning, либо еще реже - RandomForest и прочие модели(в том числе и бустинги). Причем в тех работах где используется Random Forest и бустинги обычно временные ряды "обогащены" дополнительными категориальными переменными (например температура, влажность, осадки, скорость ветра и т.д.). В нашей же работе категориальные фичи были получены из самого timeseries, что возможно и явилось причиной того, что их использование в случае линейной регрессии только ухудшало качество предсказания, а в случае Catboost лишь немного улучшило результаты.<br>
2) В нашей работе для предсказания индекса AQI были использованы предыдущие индексы усредненные по дням. Не исключено, что в случае использования часовых индексов AQI мы могли бы добиться чуть более лучших результатов. А еще возможно стоило проверить предсказание индекса на текущий день не по предыдущим индексам, а по предыдущим измеренным концентрациям загрязнителей.<br>
3) В данной работе практически никак не применялись методы обработки/подготовки данных специфичные для временных рядов. Полученный ряд не проверялся ни на автокорреляцию, ни на сезонность, ни на шумы. Вполне возможно, что дополнительная предобработка временного ряда могла бы немного улучшить результаты.<br>
4) В рамках данной работы решалась в принципе наверное очень сложная (а может даже и невозможная) задача предсказания индекса AQI по предыдущим значениям AQI. Здесь не решалась более простая задача предсказания AQI по концентрациям загрязнителей (которую как раз таки вполне можно решить, особенно бустингом).<br>
5) В качестве данных были взяты данные одной станции в Дании. Данная страна характеризуется относительно высоким уровнем качества воздуха. Возможно, в случае проведения экспериментов по измерениям станции расположенной в более загрязненном индустриальном городе, мы могли бы получить более качественную модель за счет более широкого диапазона AQI и более выраженных колебаний концентраций загрязнителей в зависимости от времени суток, дня недели, сезона, праздников и др.<br>
<br>
ВЫВОД: К сожалению, для поставленной задачи ни одна из использованных моделей не показала удовлетворительных результатов. Наилучший результат был получен на модели простой линейной регрессии. Возможно, что для последующих экспериментов целесообразнее/правильнее использовать модели Deep-Learning. Также вероятно что дополнительные данные, такие как например данные о погоде могли бы немного улучшить качество предсказания.

# [ВВЕРХ](#part_0)